# Problem might lie in loss! 
# Noooooo! Problem is not the loss! 

In [20]:
import torch
import torch.nn as nn

In [43]:
ls = nn.MSELoss(reduction='none')

In [44]:
mask = torch.tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 0.],
        [1., 0., 0.]])

In [72]:
x = torch.rand(128, 3, 2)
y = torch.rand(128, 3, 2)
mask = torch.ones(128, 3)

In [73]:
loss = ls(x, y)

In [74]:
loss.shape

torch.Size([128, 3, 2])

In [75]:
mask.shape

torch.Size([128, 3])

In [76]:
b, t, f = y.shape

In [77]:
maskex = mask.unsqueeze(-1).expand((b, t, f)).bool()

In [78]:
maskex

tensor([[[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

      

In [79]:
loss.masked_select(maskex).mean()

tensor(0.1741)

In [80]:
y_hat_masked = y.masked_select(maskex)
y_masked = x.masked_select(maskex)

In [81]:
loss_fn = nn.MSELoss(reduction='mean')

In [82]:
loss = loss_fn(y_hat_masked, y_masked)

In [83]:
loss

tensor(0.1741)

In [84]:
loss_fn = nn.MSELoss(reduction='none')

In [85]:
def masked_loss(loss_fn, y_hat, y, mask):
    """
    Computes the masked loss given a loss function, predicted values, ground truth values, and a mask tensor.
    """
    b, t, f = y_hat.shape
    mask = mask.unsqueeze(-1).expand((b, t, f)).bool()  # both y_hat and y are of same size
    # Apply mask to the predicted and ground truth values
    y_hat_masked = y_hat.masked_select(mask)
    y_masked = y.masked_select(mask)

    # Calculate the loss using the masked values
    # loss = loss_fn(y_hat_masked, y_masked)
    loss = torch.sum(loss_fn(y_hat_masked, y_masked)) / torch.sum(mask)

    return loss

In [86]:
masked_loss(loss_fn, y, x, mask)

tensor(0.1741)

In [87]:
torch.sum(mask)

tensor(384.)

In [2]:
import librosa
import librosa.display
import numpy as np

In [3]:
y, sr = librosa.load('s2201b_00000935.wav', sr = None)

In [2]:

normalizedy = librosa.util.normalize(y)

stft = librosa.core.stft(normalizedy, n_fft = 256, hop_length=16)
mel = librosa.feature.melspectrogram(S=stft, n_mels=32)
mellog = np.log(mel + 1e-9)
melnormalized = librosa.util.normalize(mellog)

In [20]:
import torchaudio

In [21]:
waveform, sample_rate = torchaudio.load('s2201b_00000935.wav', normalize=True)

In [23]:
mfcc = torchaudio.transforms.MFCC(n_mfcc=13)

In [24]:
m = mfcc(waveform)

In [26]:
m

tensor([[[-3.7962e+02, -3.6230e+02, -3.6508e+02, -3.6261e+02, -3.4902e+02,
          -3.7203e+02, -3.2691e+02, -2.9459e+02, -2.9167e+02, -2.8693e+02,
          -2.7629e+02, -2.8340e+02, -2.8161e+02, -2.8229e+02, -2.7193e+02,
          -2.7922e+02, -2.8311e+02, -2.8034e+02, -2.7894e+02, -2.8684e+02,
          -2.7806e+02, -2.7558e+02, -2.7373e+02, -2.7846e+02, -2.8148e+02,
          -2.8047e+02, -2.9212e+02, -3.0340e+02, -3.0164e+02, -3.2727e+02,
          -3.4559e+02, -3.4841e+02, -3.5718e+02, -3.5382e+02, -3.6979e+02,
          -3.6509e+02, -3.5893e+02, -3.5052e+02, -3.5979e+02, -3.4636e+02,
          -3.5895e+02, -3.6911e+02, -3.5946e+02, -3.8408e+02, -3.8175e+02,
          -3.5572e+02, -3.1451e+02, -3.3534e+02, -3.3453e+02, -2.8098e+02,
          -3.0376e+02, -2.7683e+02, -3.2178e+02, -3.3437e+02, -3.8629e+02,
          -4.4410e+02, -5.0081e+02, -5.2545e+02],
         [ 8.1613e+01,  1.0408e+02,  1.0015e+02,  1.0229e+02,  9.2200e+01,
           1.0163e+02,  1.3707e+02,  1.1183e+02,  

In [18]:
t = torchaudio.transforms.MelSpectrogram(sample_rate, n_fft=400, n_mels=64)

In [9]:
atd = torchaudio.transforms.AmplitudeToDB()

In [10]:
mel = t(waveform)

In [11]:
mel

tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.6428e-01, 5.1216e-04, 1.1408e-04,  ..., 2.9220e-01,
          3.2471e-01, 1.2320e-01],
         [8.8451e-01, 2.7576e-03, 6.1423e-04,  ..., 1.5733e+00,
          1.7484e+00, 6.6336e-01],
         ...,
         [1.8699e-06, 3.0843e-06, 5.5021e-06,  ..., 5.5968e-06,
          2.0315e-06, 1.4247e-06],
         [9.3079e-06, 4.7386e-06, 3.8569e-06,  ..., 3.1327e-06,
          8.1938e-07, 4.1661e-07],
         [4.6462e-06, 4.3422e-06, 1.6264e-06,  ..., 2.0404e-06,
          7.7219e-07, 3.5185e-07]]])

In [12]:
mel.shape

torch.Size([1, 128, 58])

In [13]:
dbmel = atd(mel)

In [14]:
dbmel

tensor([[[-100.0000, -100.0000, -100.0000,  ..., -100.0000, -100.0000,
          -100.0000],
         [  -7.8443,  -32.9060,  -39.4280,  ...,   -5.3432,   -4.8850,
            -9.0938],
         [  -0.5330,  -25.5947,  -32.1167,  ...,    1.9681,    2.4263,
            -1.7825],
         ...,
         [ -57.2818,  -55.1084,  -52.5947,  ...,  -52.5206,  -56.9218,
           -58.4627],
         [ -50.3115,  -53.2435,  -54.1376,  ...,  -55.0409,  -60.8652,
           -63.8027],
         [ -53.3290,  -53.6229,  -57.8876,  ...,  -56.9029,  -61.1228,
           -64.5364]]])

In [15]:
dbmel.shape

torch.Size([1, 128, 58])